Secure Data Vault (SDV)
==========

This notebook is used to test drive **SDV**, an open source library from MIT for generating synthetic data.

Data Set
------------------

[Lending Club loan data set](https://www.kaggle.com/wordsforthewise/lending-club).  I like this data set due to its size and high dimensionality.  The data inside is also pretty raw with many columns containing null values which provides us an opportunity to do some data prep/cleansing - just like in any real-world situation.

Many of SDV's models are computationally intensive and will overwhelm most general-purpose VM instances (e.g. Azure's Dv4-series).  I've created a utility called [randomSampler.py](./randomSampler.py) to help reduce the data set to a more manageable size for your personal equipment.

### Load example data


In [2]:
import pandas as pd

original_df = pd.read_csv('./500_accepted_2007_2018Q4.csv', low_memory=False)

Let's create a copy of the original dataframe.  We'll play with the copy to avoid having to reload the original dataset from disk.


In [3]:
df = original_df.copy(deep=True)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,67398322,NaN,21000.0,21000.0,21000.0,36 months,8.49,662.83,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68102522,NaN,24000.0,24000.0,24000.0,36 months,9.80,772.17,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,67357364,NaN,26000.0,26000.0,26000.0,36 months,13.67,884.46,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,67215673,NaN,26000.0,26000.0,26000.0,36 months,7.26,805.90,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,66575072,NaN,4200.0,4200.0,4200.0,36 months,7.89,131.40,A,A5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


Check dimensionality.  In this case, how many (rows, columns) are we working with?

In [4]:
df.shape

(500, 151)

Check a few of the columns.

In [5]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)

## Exploratory data analysis

Are there any columns with missing values?

In [14]:
# Find the count and percentage of missing values
df_na = pd.DataFrame({
    '% Empty': 100*df.isnull().sum()/len(df), 
    'Count': df.isnull().sum()
    })

# Print columns with null count > 0
df_na[df_na['Count'] > 0]

#print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
#    "There are " + str(df_na[df_na['Count'] >0]) +
#    " columns that have missing values.")

,% Empty,Count
member_id,100.0,500
emp_title,5.8,29
emp_length,5.0,25
desc,94.6,473
title,0.6,3
...,...,...
settlement_status,98.6,493
settlement_date,98.6,493
settlement_amount,98.6,493
settlement_percentage,98.6,493


SDV GaussianCopula
---------------------------------


In [15]:
from sdv.tabular import GaussianCopula

copula_model = GaussianCopula()
copula_model.fit(df)

In [ ]:
copula_model.save('lending_club_copula_model.pkl')

In [ ]:
loaded_model = GaussianCopula.load('lending_club_copula_model.pkl')

### Generate synthetic data from model

In [16]:
synthetic_data = copula_model.sample(200)

In [17]:
synthetic_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,63934077,NaN,6917.090478,7431.013537,7440.417082,36 months,14.146686,266.444437,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,62223791,NaN,3126.766591,3218.898829,3148.941281,36 months,25.149506,87.838224,B,C2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,34195192,NaN,6831.773089,7233.606526,6772.272787,36 months,16.510359,270.105475,E,A1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,171265080,NaN,8467.200829,8174.217091,8424.713262,36 months,10.155162,305.300025,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,81230441,NaN,20641.881174,20757.326497,20659.329426,36 months,9.752449,565.814135,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
